In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
# !pip install -U llama-index

In [3]:
from utils import load_env

load_env()

# Designing the Workflow TBD

RAG + Reranking consists of some clearly defined steps

Indexing data, creating an index
Using that index + a query to retrieve relevant text chunks
Rerank the text retrieved text chunks using the original query
Synthesizing a final response
With this in mind, we can create events and workflow steps to follow this process!

In [4]:
# TBD
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore


class RetrieverEvent(Event):
    """Result of running retrieval"""

    nodes: list[NodeWithScore]


class RerankEvent(Event):
    """Result of running reranking on retrieved nodes"""

    nodes: list[NodeWithScore]

# Workflow components

## local neo4j docker instance


for mac or linux, use below
```bash
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```

for windows on anaconda prompt, run below

```bash
docker run ^
   -p 7474:7474 -p 7687:7687 ^
   -v "%CD%/data:/data" -v "%CD%/plugins:/plugins" ^
   --name neo4j-apoc ^
   -e NEO4J_apoc_export_file_enabled=true ^
   -e NEO4J_apoc_import_file_enabled=true ^
   -e NEO4J_apoc_import_file_use__neo4j__config=true ^
   -e NEO4JLABS_PLUGINS="[\"apoc\"]" ^
   neo4j:latest
```
Go see your instance at http://localhost:7474/browser/. Default login and password is 'neo4j'.

You will be asked to change the password, change it to 'password'.


In [5]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

username = "neo4j"
password = "password"
url = "bolt://localhost:7687"

# connect to the graph store
graph_store = Neo4jPGStore(username=username, password=password, url=url)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


## The Workflow Itself
With our events defined, we can construct our workflow and steps

Note that the workflow automatically validates itself using type annotations, so the type annotations on our steps are very helpful

In [6]:
from llama_index.core.schema import TextNode
from typing import List
import json

# helper functions
def get_text_nodes(json_list: List[dict]):
    text_nodes = []
    for idx, page in enumerate(json_list):
        text_node = TextNode(text=page["md"], metadata={"page": page["page"]})
        text_nodes.append(text_node)
    return text_nodes


def save_jsonl(data_list, filename):
    """Save a list of dictionaries as JSON Lines."""
    with open(filename, "w") as file:
        for item in data_list:
            json.dump(item, file)
            file.write("\n")


def load_jsonl(filename):
    """Load a list of dictionaries from JSON Lines."""
    data_list = []
    with open(filename, "r") as file:
        for line in file:
            data_list.append(json.loads(line))
    return data_list

In [7]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import SimpleDirectoryReader


class GraphRAGWorkflow(Workflow):
    @step(pass_context=True)
    async def ingest_docs(self, ctx: Context, ev: StartEvent) -> StopEvent | None:
        """Ingest the documents using LlamaParse"""
        dirpath = ev.get("dirpath")
        if not dirpath:
            return None
        
        docs = SimpleDirectoryReader(dirpath).load_data()
        print(f"ingested {len(docs)} docs")
        return StopEvent(result=docs)
        # return StopEvent(result=f"ingested {len(docs)} docs")



    @step(pass_context=True)
    async def index_docs(self, ctx: Context, ev: StartEvent) -> StopEvent | None:
        """Index the documents dynamically"""
        max_triplets = ev.get("max_triplets")
        if not max_triplets:
            return None
        
        docs: List[TextNode] = ctx.data.get("docs", [])
        kg_extractor = DynamicLLMPathExtractor(
            llm=llm,
            max_triplets_per_chunk=max_triplets,
            num_workers=4,
            allowed_entity_types=None,
            allowed_relation_types=None,
            allowed_relation_props=None,
            allowed_entity_props=None,
        )

        dynamic_index = PropertyGraphIndex.from_documents(
            docs,
            llm=llm,
            embed_model=embed_model,
            property_graph_store=graph_store,
            kg_extractors=[kg_extractor],
            show_progress=True,
        )

        return StopEvent(result=dynamic_index)

        

# Run the workflow

In [9]:
# Ingest the documents

w = GraphRAGWorkflow(timeout=120, verbose=True)
docs = await w.run(dirpath="data/kg_rag_workflow/")

Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 86 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 127 0 (offset 0)
Ignoring wrong pointing object 140 0 (offset 0)
Ignoring wrong pointing object 180 0 (offset 0)


Running step index_docs
Step index_docs produced no event
Running step ingest_docs
ingested 46 docs
Step ingest_docs produced event StopEvent


In [18]:
# Display the first 2 documents
print(f"docs is type {type(docs[0])}")
for doc in docs[:2]:
    print("-" * 50)
    print(doc)

docs is type <class 'llama_index.core.schema.Document'>
--------------------------------------------------
Doc ID: 349cafba-c028-4bf4-a27a-ae90183322f9
Text: 139 Archivio Italiano di Urologia e Andrologia 2019; 91, 3REVIEW
Nutraceutical treatment and prevention  of benign prostatic
hyperplasia and prostate cancer  Arrigo F.G. Cicero1, Olta
Allkanjari2, Gian Maria Busetto3, Tommaso Cai4, Gaetano Larganà5,
Vittorio Magri6, Gianpaolo Perletti7, Francesco Saverio Robustelli
Della Cuna8, Giorgio Ivan Russ...
--------------------------------------------------
Doc ID: 84232ada-e846-4d26-ad74-dd321ead2b27
Text: Archivio Italiano di Urologia e Andrologia 2019; 91, 3A.F.G.
Cicero, O. Allkanjari, G.M. Busetto, et al.
140INTRODUCTIONTONUTRACEUTICALPRESCRIPTION (Arrigo F.G. Cicero) During
the last years, the pharmaceutical innovation in primary care are
dramatically less frequent and will be even more rare in the next
future. In this context, pre- clinical...


In [11]:
# dynamic_index = w.run(max_triplets=20, timeout=120, verbose=True)

In [19]:
len(docs)
type(docs[0])

llama_index.core.schema.Document

In [28]:
len(docs[0].text)

5866

In [29]:
print(docs[0].text)

139 Archivio Italiano di Urologia e Andrologia 2019; 91, 3REVIEW
Nutraceutical treatment and prevention 
of benign prostatic hyperplasia and prostate cancer 
Arrigo F.G. Cicero1, Olta Allkanjari2, Gian Maria Busetto3, Tommaso Cai4, Gaetano Larganà5,
Vittorio Magri6, Gianpaolo Perletti7, Francesco Saverio Robustelli Della Cuna8, Giorgio Ivan Russo5,
Kostantinos Stamatiou9, Alberto Trinchieri10, Annabella Vitalone2
1 Dip. di Scienze Mediche e Chirurgiche, Alma Mater Studiorum Università di Bologna, Bologna, Italy;
2 Dipartimento di Farmacologia e Fisiologia “V. Erspamer”, Sapienza, Università di Roma, Roma, Italy;
3 Department of Urology, Sapienza Università di Roma, Policlinico Umberto I, Roma, Italy;
4 Department of Urology, Santa Chiara Regional Hospital, Trento, Italy;
5 Urology Department, University of Catania, Catania, Italy;
6 Ambulatorio Territoriale di Urologia ed Ecograﬁa Urologica, ASST Nord Milano, Milano, Italy;
7 Dipartimento di Biotecnologie e Scienze della Vita, Sezione 

In [30]:
docs[0].metadata

{'page_label': '1',
 'file_name': 'bph-review-2019.pdf',
 'file_path': '/Users/michaelkoch/github/posts/data/kg_rag_workflow/bph-review-2019.pdf',
 'file_type': 'application/pdf',
 'file_size': 1558509,
 'creation_date': '2024-08-03',
 'last_modified_date': '2024-08-02'}

In [25]:
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor
from llama_index.core import PropertyGraphIndex

llm = OpenAI(model="gpt-4o-mini", temperature=0.0)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")


kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=None,
    allowed_relation_types=None,
    allowed_relation_props=[],
    allowed_entity_props=[],
    # allowed_entity_types=["MEDICINAL_PLANT", "COMPOUND", "SYMPTOM", "TREATMENT", "DISEASE", "STUDY_TYPE"],
    # allowed_relation_types=["TREATS", "CONTAINS", "ALLEVIATES", "STUDIED_IN", "SIDE_EFFECT_OF", "INTERACTS_WITH"],
    # allowed_relation_props=["efficacy", "dosage"],
    # allowed_entity_props=["scientific_name", "common_name", "description"]
)

dynamic_index = PropertyGraphIndex.from_documents(
            [docs[0]],
            llm=llm,
            embed_model=embed_model,
            property_graph_store=graph_store,
            kg_extractors=[kg_extractor],
            show_progress=True,
        )

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 281.19it/s]
Extracting and inferring knowledge graph from text: 100%|██████████| 2/2 [00:22<00:00, 11.26s/it]
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.27it/s]
Generating embeddings: 0it [00:00, ?it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
